Table 4.12.2: Number of children living with individuals starting treatment 2017-18

In [1]:
from gssutils import *

if is_interactive():
    scraper = Scraper('https://www.gov.uk/government/collections/alcohol-and-drug-misuse-and-treatment-statistics')
    scraper.select_dataset(title=lambda x: x.startswith('Substance misuse treatment for adults'), latest=True)
    tabs = {tab.name: tab for tab in scraper.distribution(title=lambda x: x.startswith('Data tables')).as_databaker()}

In [2]:
tab = tabs['Table 4.12.2']

https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/752515/AdultSubstanceMisuseNDTMSDataTables2017-18.xlsx

In [3]:
cell = tab.filter('Number of children')
observations = tab.filter('Total').fill(DOWN).is_not_blank().is_not_whitespace().is_number()
children = cell.expand(DOWN).is_not_blank().is_not_whitespace()
Clients = cell.expand(RIGHT).is_not_blank().is_not_whitespace().is_not_number()
Dimensions = [
            HDim(children,'Treatment group',DIRECTLY,LEFT),
            HDim(Clients,'Clients in treatment',CLOSEST,LEFT),
            HDimConst('Measure Type','count-of-children'),
            HDimConst('Unit','children'),
            HDimConst('Period', 'gregorian-interval/2017-04-01T00:00:00/P1Y')
            ]

In [4]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
if is_interactive():
    savepreviewhtml(c1)

In [5]:
new_table = c1.topandas()

In [6]:
import numpy as np
new_table['OBS'].replace('', np.nan, inplace=True)
new_table.dropna(subset=['OBS'], inplace=True)
new_table.rename(columns={'OBS': 'Value'}, inplace=True)
new_table['Value'] = new_table['Value'].astype(int)
new_table['Value'] = new_table['Value'].map(lambda x:''
                                  if (x == '-')
                                  else int(x))

In [7]:
new_table['Basis of treatment'] = new_table['Treatment group'].map(
    lambda x: {
        'Number of children':'number-of-children',
        'Number of children':'number-of-children/number-of-children',
        'Number of individuals living with children':'number-of-children/number-of-individuals-living-with-children',
        'Average number of children living with people in treatment':'number-of-children/average-number-of-children-living-with-people-in-treatment',

        }.get(x, x))

In [8]:
import urllib.request as request
import csv
import io
import requests
vrl="https://raw.githubusercontent.com/ONS-OpenData/ref_alcohol/master/codelists/substance-type.csv"
t=requests.get(vrl).content
g=pd.read_csv(io.StringIO(t.decode('utf-8')))
new_table = pd.merge(new_table, g, how = 'left', left_on = 'Clients in treatment', right_on = 'Label')
new_table.columns = ['Substance type' if x=='Notation' else x for x in new_table.columns]
new_table = new_table[['Period','Basis of treatment','Substance type','Measure Type','Value','Unit']]

In [9]:
new_table.head()

,Period,Basis of treatment,Substance type,Measure Type,Value,Unit
0,gregorian-interval/2017-04-01T00:00:00/P1Y,number-of-children/number-of-children,opiate,Count,10328,children
1,gregorian-interval/2017-04-01T00:00:00/P1Y,number-of-children/number-of-children,non-opiate-only,Count,7424,children
2,gregorian-interval/2017-04-01T00:00:00/P1Y,number-of-children/number-of-children,non-opiate-and-alcohol,Count,7453,children
3,gregorian-interval/2017-04-01T00:00:00/P1Y,number-of-children/number-of-children,alcohol-only,Count,20904,children
4,gregorian-interval/2017-04-01T00:00:00/P1Y,number-of-children/number-of-children,total,Count,46109,children
